In [53]:
import cv2
import numpy as np

Загрузка изображения

In [54]:
img_path = '/home/evgen/Coursework/dct_compress/fisherman-4465032_960_720.jpg' # fisherman-4465032_960_720.jpg

img = cv2.imread(img_path, cv2.IMREAD_COLOR) #COLOR_YCrCb2BGR cv2.COLOR_BGR2YCR_CB)

if img is None:
    print("Ошибка: изображение не загружено. Проверьте путь к файлу.")
else:
    # Преобразование в цветовое пространство YCrCb
    img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    # cv2.imshow('Image in YCrCb', img_ycrcb)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


Предобработка

In [55]:
# Приведение изображения к кратным размерам
def pad_image_to_multiple(img, block_size = 8):
    h, w, channels = img.shape

    # pad block size
    pad_h = (block_size - h % block_size) % block_size
    pad_w = (block_size - w % block_size) % block_size
    # print(pad_h, pad_w)

    new_h = h + pad_h
    new_w = w + pad_w

    new_img = np.zeros((new_h, new_w, channels), dtype=np.uint8)
    new_img[:h, :w] = img

    return new_img



In [56]:
def split_into_blocs(img, block_size = 8):
    h, w, channels = img.shape

    blocks = img.reshape(h // block_size, block_size,
                         w // block_size, block_size, 
                         channels)
    

    #Меняем оси для правильной структуры блоков
    blocks = blocks.swapaxes(1, 2)
    return blocks

Применение "дискретно-косинусного преобразования"

In [57]:
# DCT
def dct2D(block):

    """
    Реализация 2D DCT для блока N x N.
    :param block: входной блок размером N x N
    :return: коэффициенты DCT размером N x N
    """

    N = block.shape[0]
    dct_res = np.zeros((N, N), dtype=np.float32)
    alpha = lambda k : np.sqrt(1/N) if k == 0 else np.sqrt(2/N)

    for u in range(N):
        for v in range(N):
            sum_value = 0.0
            for x in range(N):
                for y in range(N):
                    sum_value += block[x, y] * \
                                 np.cos((2 * x + 1) * u * np.pi / (2 * N)) * \
                                 np.cos((2 * y + 1) * v * np.pi / (2 * N))
            dct_res[u, v] = alpha(u) * alpha(v) * sum_value

    return dct_res

In [58]:
def apply_dct_to_blocks(blocks):
    """
    Применить 2D DCT к каждому блоку изображения.
    :param blocks: блоки изображения размером (H, W, block_size, block_size, channels)
    :return: коэффициенты DCT для каждого блока
    """
    H, W, block_size, _, channels = blocks.shape
    dct_blocks = np.zeros_like(blocks, dtype=np.float32)

    # Применяем DCT к каждому блоку для каждого канала
    for i in range(H):
        for j in range(W):
            for c in range(channels):
                block = blocks[i, j, :, :, c]  # Извлекаем блок для текущего канала
                dct_blocks[i, j, :, :, c] = dct2D(block)

    return dct_blocks

In [59]:
new_im = pad_image_to_multiple(img_ycrcb, 8)
blocks = split_into_blocs(new_im, 8)
res = apply_dct_to_blocks(blocks)

Квантование коэффициентов

In [60]:
# Пример матриц квантования для каждого канала
quant_matrices = [
    np.array([  # Матрица для Y (яркость)
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ]),
    np.array([  # Матрица для Cb (синий компонент цветности)
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99]
    ]),
    np.array([  # Матрица для Cr (красный компонент цветности)
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99]
    ])
]

In [61]:
# def quantize_dct_blocks(dct_blocks, quant_matrix):
#     """
#     Применить квантование к каждому блоку коэффициентов DCT.
#     :param dct_blocks: массив коэффициентов DCT (H, W, block_size, block_size, channels)
#     :param quant_matrix: матрица квантования коэффициентов (block_size x block_size)
#     :return: квантизированные коэффициенты
#     """
#     H, W, block_size, _, channels = dct_blocks.shape
#     quantized_blocks = np.zeros_like(dct_blocks, dtype=np.int32)

#     for i in range(H):
#         for j in range(W):
#             for c in range(channels):
#                 quantized_blocks[i, j, :, :, c] = np.round(dct_blocks[i, j, :, :, c] / quant_matrix)

#     return quantized_blocks

def quantize_dct_blocks_multi_channel(dct_blocks, quant_matrices):
    """
    Применить квантование к каждому блоку коэффициентов DCT с разными матрицами для каждого канала.
    :param dct_blocks: массив коэффициентов DCT (H, W, block_size, block_size, channels)
    :param quant_matrices: список матриц квантования для каждого канала [(block_size x block_size), ...]
    :return: квантизированные коэффициенты
    """
    H, W, block_size, _, channels = dct_blocks.shape
    quantized_blocks = np.zeros_like(dct_blocks, dtype=np.int32)

    for c in range(channels):
        for i in range(H):
            for j in range(W):
                quantized_blocks[i, j, :, :, c] = np.round(dct_blocks[i, j, :, :, c] / quant_matrices[c])

    return quantized_blocks

# Квантуем коэффициенты
quantized_dct_blocks = quantize_dct_blocks_multi_channel(res, quant_matrices)

In [62]:
print(quantized_dct_blocks.shape)

(90, 60, 8, 8, 3)


Дальнейшая обработка коэффициентов

In [66]:
def zigzag_scan(block):
    """
    Преобразовать 2D-блок в зигзагообразный порядок.
    :param block: 2D массив (block_size x block_size)
    :return: 1D массив значений в зигзагообразном порядке
    """
    block_size = block.shape[0]
    result = []
    for i in range(2 * block_size - 1):
        if i % 2 == 0:
            for x in range(max(0, i - block_size + 1), min(i + 1, block_size)):
                result.append(block[x, i - x])
        else:
            for x in range(max(0, i - block_size + 1), min(i + 1, block_size)):
                result.append(block[i - x, x])
    return result


def zigzag_transform_blocks(quantized_blocks):
    """
    Преобразовать все блоки в зигзагообразный порядок.
    :param quantized_blocks: массив квантованных коэффициентов (H, W, block_size, block_size, channels)
    :return: массив зигзагообразных преобразований для каждого блока
    """
    H, W, block_size, _, channels = quantized_blocks.shape
    zigzag_blocks = np.zeros((H, W, block_size * block_size, channels), dtype=np.int32)

    for c in range(channels):
        for i in range(H):
            for j in range(W):
                zigzag_blocks[i, j, :, c] = zigzag_scan(quantized_blocks[i, j, :, :, c])

    return zigzag_blocks

In [69]:
# Преобразуем квантованные блоки
zigzag_blocks = zigzag_transform_blocks(quantized_dct_blocks)
print("Зигзаг для первого блока канала Y:", zigzag_blocks[0, 0, :, 1])

Зигзаг для первого блока канала Y: [63  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


компрес RLE

In [70]:
def rle_encode(array):
    """
    Применить RLE-кодирование к массиву.
    :param array: входной одномерный массив
    :return: массив пар (значение, количество)
    """
    encoded = []
    prev_value = array[0]
    count = 1
    for value in array[1:]:
        if value == prev_value:
            count += 1
        else:
            encoded.append((prev_value, count))
            prev_value = value
            count = 1
    encoded.append((prev_value, count))  # Добавить последний элемент
    return encoded

# Пример кодирования одного блока
# zigzag_flatten = zigzag_blocks[0, 0, :, 0]  # Первый блок первого канала
# rle_result = rle_encode(zigzag_flatten)
# print(rle_result)



def apply_rle_to_all_blocks(zigzag_blocks):
    """
    Применить RLE-кодирование ко всем зигзагообразным блокам.
    :param zigzag_blocks: массив зигзагообразных преобразований (H, W, block_size^2, channels)
    :return: массив RLE-кодированных блоков
    """
    H, W, block_size_sq, channels = zigzag_blocks.shape
    rle_encoded_blocks = np.empty((H, W, channels), dtype=object)

    for c in range(channels):
        for i in range(H):
            for j in range(W):
                rle_encoded_blocks[i, j, c] = rle_encode(zigzag_blocks[i, j, :, c])

    return rle_encoded_blocks

# Применение RLE ко всем блокам
rle_encoded_blocks = apply_rle_to_all_blocks(zigzag_blocks)

# Пример: вывод результата для первого блока первого канала
print(rle_encoded_blocks)

[[[list([(71, 1), (0, 63)]) list([(63, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(71, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(72, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  ...
  [list([(74, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(74, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(73, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]]

 [[list([(71, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(72, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(72, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  ...
  [list([(74, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(74, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]
  [list([(73, 1), (0, 63)]) list([(62, 1), (0, 63)])
   list([(56, 1), (0, 63)])]]



Декомпрессия RLE

In [72]:
def rle_decode(rle_data):
    """
    Декодировать массив, закодированный с помощью RLE.
    :param rle_data: массив пар (значение, количество)
    :return: декодированный 1D массив
    """
    decoded = []
    for value, count in rle_data:
        decoded.extend([value] * count)
    return decoded

In [73]:
def decode_rle_from_all_blocks(rle_encoded_blocks):
    """
    Применить декодирование RLE ко всем блокам.
    :param rle_encoded_blocks: массив RLE-кодированных блоков (H, W, channels)
    :return: массив зигзагообразных блоков (H, W, block_size^2, channels)
    """
    H, W, channels = rle_encoded_blocks.shape
    # Предполагаем, что блокы квадратные и имеют одинаковый размер
    # Восстанавливаем размерность на основе количества элементов в одном блоке
    block_size_sq = len(rle_decode(rle_encoded_blocks[0, 0, 0]))
    zigzag_blocks = np.zeros((H, W, block_size_sq, channels), dtype=np.int32)

    for c in range(channels):
        for i in range(H):
            for j in range(W):
                zigzag_blocks[i, j, :, c] = rle_decode(rle_encoded_blocks[i, j, c])

    return zigzag_blocks


In [74]:
# Декодируем все блоки
decoded_zigzag_blocks = decode_rle_from_all_blocks(rle_encoded_blocks)

# Проверим восстановление первого блока первого канала
print("Декодированный первый блок (первый канал):")
print(decoded_zigzag_blocks[0, 0, :, 0])

Декодированный первый блок (первый канал):
[71  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


Визуализация паттернов высокой и низкой частоты, отображающие энергию и структуру каждого блока.

In [37]:
def normalize_block(block):

    block = np.abs(block)
    norm_block = 255 * (block - block.min() / (block.max() - block.min()))
    return norm_block

def reconstruct_img_frec(blocks):

    H, W, block_size, _, channals = blocks.shape

    h = H * block_size
    w = W * block_size

    image = np.zeros((h, w, channals), dtype=np.uint8)

    for i in range(H):
        for j in range(W):
            for c in range(channals):
                block = blocks[i, j, :, :, c]
                norm_block = normalize_block(block)
                image[i * block_size : (i + 1) * block_size,
                      j * block_size : (j + 1) * block_size, c] = norm_block
                
    return image
                

In [38]:
img_dtc = reconstruct_img_frec(res)
cv2.imshow("DCT Image", img_dtc)
cv2.waitKey(0)
cv2.destroyAllWindows()